# 01-01 Extraktion Demo

## Konfiguration des Notebooks

In [ ]:
# Ggf. fehlende Pakete installieren
!pip install --quiet ipython-sql petl

In [ ]:
import petl as etl
import csv
import sys
%load_ext sql

In [ ]:
# Konfiguration
base_url_quellen = "https://raw.githubusercontent.com/fau-lmi/lct-ehealth/main/07-ETL+DWH/data"
base_url_staging = "./"

**Hinweise zur Datenbankanbindung**

Im folgenden Block werden die Verbindungen zur Datenbank hergestellt. Normalerweise würden wir für das Staging eine einzige Zieldatenbank nutzen, in der wir verschiedene Schemata als Namensräume für die 3 Quellsysteme (Bielefeld, Mannheim, Dimensionen) nutzen würden. Die Datenbanken werden dabei als `*.sqlite`-Dateien im aktuellen Verzeichnis des Rechners angelegt, auf dem dieses Notebook ausgeführt wird.

In [ ]:
# Datenbankverbindung als Pfad (für das ETL) & iPython SQL (für die Abfragen) herstellen
db_path_stg_bielefeld  = base_url_staging + "stg_bielefeld.sqlite"
db_path_stg_mannheim   = base_url_staging + "stg_mannheim.sqlite"
db_path_stg_dimensions = base_url_staging + "stg_dimensions.sqlite"
db_path_staging        = base_url_staging + "staging.sqlite"

db_url_stg_bielefeld  = "sqlite:///" + db_path_stg_bielefeld
db_url_stg_mannheim   = "sqlite:///" + db_path_stg_mannheim
db_url_stg_dimensions = "sqlite:///" + db_path_stg_dimensions
db_url_staging        = "sqlite:///" + db_path_staging

%sql $db_url_stg_bielefeld
%sql $db_url_stg_mannheim
%sql $db_url_stg_dimensions
%sql $db_url_staging

Da SQLite keine Schemata unterstützt, legen wir stattdessen 3 getrennte Staging-Datenbanken an (stg_bielefeld, stg_mannheim & stg_dimensions). SQLite erlaubt über den `ATTACH DATABASE`-Befehl, diese 3 Datenbanken anschließend von einer gemeinsamen Datenbank (staging) aus zu bündeln und gemeinsam darauf zuzugreifen.

Wir können  anschließend von der übergeordneten Staging-Datenbank aus gleichzeitig auf Tabellen jeder der 3 eingehängten Datenbanken zugreifen, indem wir z.B. `SELECT * FROM stg_bielefeld.faelle` schreiben (lädt die Daten der Fälle-Tabelle der Bielefelder Staging-Datenbank).

In [ ]:
%%sql $db_url_staging
ATTACH DATABASE :db_path_stg_bielefeld  AS bielefeld;
ATTACH DATABASE :db_path_stg_mannheim   AS mannheim;
ATTACH DATABASE :db_path_stg_dimensions AS dimensions;

## Rohdatentabelle extrahieren und in Staging-Bereich laden

In den folgenden Blocks nutzen wir das PETL-Paket, um mit wenigen Befehlen die Tabelle 'standorte.tsv' aus den Dimensions-Rohdaten in die Staging-Datenbank stg_dimensions zu übertragen.

### 1. Zieltabelle anlegen

Hierzu legen wir zunächst mit einem CREATE-Table die Tabelle in der Staging-Datenbank (leer) an.

💡 *Hinweis:* Der Code zur Anlage der Tabelle muss in einer eigenen Zelle stehen, da der `%%sql`-Befehl für mehrzeilige SQL-Statements nicht gemeinsam mit anderen Befehlen im gleichen Block verarbeitet werden kann.

In [ ]:
%%sql $db_url_staging
DROP TABLE IF EXISTS dimensions.standorte;
CREATE TABLE dimensions.standorte (
  standort_id   VARCHAR(1),
  standort_name VARCHAR(20)
);

### 2. Daten aus Quelldatei auslesen & in Zieltabelle schreiben

Als nächstes laden wir mit der PETL-Funktion `fromtsv()` die Quelldaten aus der Datei 'standorte.tsv' und speichern diese direkt anschließend mit der PETL-Funktion `todb()` in die Zieldatenbank stg_dimensions.

Bei der Funktion `fromtsv()` muss im Regelfall nur der Pfad/Dateiname der zu ladenden Datei angegeben werden. Alternativ kann mit `fromcsv()` auch eine Komma-getrennte Datei geladen bzw. mit dem Parameter `delimiter=";"` ein anderes Trennzeichen vorgegeben werden.

Bei der Funktion `todb()` müssen die geladene Datentabelle, die Datenbankverbindung und der Name der Tabelle in der Zieldatenbank angegeben werden. Optional kann mit dem Parameter `create=True` angegeben werden, dass die Tabelle automatisch in der Zieldatenbank angelegt werden soll. Hierbei werden allerdings (bei zuvor aus TSV/CSV-Dateien geladenen Rohdaten) nicht immer die korrekten Datentypen verwendet. Es ist deshalb sinnvoll, die Tabelle vorher manuell per `CREATE TABLE`-Statement anzulegen. Die Zieltabelle wird von der Funktion zunächst geleert und dann mit den Inhalten neu beladen. Wenn die vorhandenen Inhalte bestehen bleiben sollen, muss statt `todb()` die PETL-Funktion `appenddb()` genutzt werden.

Dokumentation zu PETL ist hier verfügbar: https://petl.readthedocs.io/en/stable/io.html

In [ ]:
table_data = etl.fromtsv(base_url_quellen + "/dimensionen/standorte.tsv")
etl.todb(table_data, db_path_stg_dimensions, 'standorte')

### 3. Ergebniskontrolle

Anschließend können wir mit einer `SELECT`-Abfrage überprüfen, ob die Tabelle korrekt übertragen wurde.

💡 *Tipp:* Wenn Sie mit einer `SELECT * FROM tabelle`-Abfrage prüfen wollen, ob Inhalte vorhanden sind, können ggf. sehr große Datenmengen in das Colab-Notebook geladen werden, was die Laufzeitumgebung bzw. den Browser zum Absturz bringen kann. Ergänzen Sie deshalb immer eine `LIMIT <Zahl>`-Klausel am Ende der Abfrage, so dass nur eine feste Zahl von Datensätzen geladen wird.

In [ ]:
%%sql $db_url_staging
SELECT *
  FROM dimensions.standorte
 LIMIT 10